In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from keras import backend as K

In [18]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [19]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

In [20]:
def model_performance(y_test, y_pred):    
    print('confusion matrix')
    print(metrics.confusion_matrix(y_test, y_pred))
    print('accuracy : {}'.format(metrics.accuracy_score(y_test, y_pred).round(3)))
    print('precision : {}'.format(metrics.precision_score(y_test, y_pred, pos_label=1).round(3)))
    print('recall : {}'.format(metrics.recall_score(y_test, y_pred, pos_label=1).round(3)))
    print('F1 : {}'.format(metrics.f1_score(y_test, y_pred, pos_label=1).round(3)))

## import data

In [21]:
cust_data = pd.read_csv('data_transform.csv')
cust_data.shape

(100233, 133)

In [22]:
# 전체 연체자 수
cust_overdue = cust_data[cust_data['TARGET'] == 1]     # 연체자
print(cust_overdue.shape[0])

4287


In [23]:
# train / test set 분리
x = cust_data.drop('TARGET', axis=1).as_matrix()
y = cust_data['TARGET']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

print('연체자 수 train : {} / {}'.format(sum(y_train == 1), y_train.shape[0]))
print('연체자 수 test  : {} / {}'.format(sum(y_test  == 1), y_test.shape[0]))

연체자 수 train : 3439 / 80186
연체자 수 test  : 848 / 20047


In [24]:
scaler = MinMaxScaler(feature_range=(0, 1))
x = scaler.fit_transform(x)
x[0]

array([ 0.42857143,  0.6       ,  0.06000188,  0.16393443,  1.        ,
        0.25947368,  0.045     ,  0.        ,  0.42857201,  0.3       ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.01953418,  0.18181818,  0.71428571,  0.        ,  0.00826446,
        0.        ,  0.        ,  0.        ,  0.10743802,  0.        ,
        0.54      ,  0.00402495,  0.0036483 ,  0.        ,  0.0475    ,
        0.        ,  0.        ,  0.02      ,  0.        ,  0.0475    ,
        0.00905632,  0.385     ,  0.4       ,  0.05263158,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.019     ,  0.        ,  0.66666667,  0.52727273,  0.08421053,
        0.        ,  0.        ,  0.        ,  0.12      ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.77631579,  0.        ,  0.        ,  0.05309735,  0.  

In [26]:
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

print(y_train.shape)

n_cols = x_train.shape[1]
print(n_cols)

(80186, 2)
132


In [25]:
# train set 중에서 연체자만 추출
x_overdue = x_train[y_train == 1]
y_overdue = y_train[y_train == 1]
print(x_overdue.shape)
print(y_overdue.shape)

(3439, 132)
(3439,)


In [27]:
print(sum(y_train[:,1]))
print(sum(y_test[:,1]))   # 연체자

3439.0
848.0


In [66]:
model = Sequential()
model.add(Dense(n_cols, activation='relu', input_shape=(n_cols,)))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [67]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[f1])

In [68]:
model.fit(x_train, y_train,
            batch_size = 1000,
            epochs = 10,
            verbose = 1,
            validation_split=0.1)

y_pred = model.predict(x_test)
print(len(y_pred[(y_pred[:,0] > y_pred[:,1])]))
print(len(y_pred[(y_pred[:,0] < y_pred[:,1])]))   # 연체자

Train on 72167 samples, validate on 8019 samples
Epoch 1/10
72167/72167 [==============================] - 2s - loss: 0.9905 - f1: 0.9205 - val_loss: 0.7417 - val_f1: 0.9540
Epoch 2/10
72167/72167 [==============================] - 1s - loss: 0.6852 - f1: 0.9575 - val_loss: 0.7409 - val_f1: 0.9540
Epoch 3/10
72167/72167 [==============================] - 1s - loss: 0.6819 - f1: 0.9556 - val_loss: 0.7417 - val_f1: 0.9540
Epoch 4/10
72167/72167 [==============================] - 1s - loss: 0.6833 - f1: 0.9575 - val_loss: 0.6760 - val_f1: 0.9540
Epoch 5/10
72167/72167 [==============================] - 1s - loss: 0.3332 - f1: 0.9572 - val_loss: 0.2576 - val_f1: 0.9540
Epoch 6/10
72167/72167 [==============================] - 1s - loss: 0.2117 - f1: 0.9572 - val_loss: 0.2233 - val_f1: 0.9540
Epoch 7/10
72167/72167 [==============================] - 1s - loss: 0.1897 - f1: 0.9573 - val_loss: 0.1901 - val_f1: 0.9540
Epoch 8/10
72167/72167 [==============================] - 1s - loss: 0.1775 

In [61]:
len(y_pred[(y_pred[:,0] > y_pred[:,1])])

20047

In [54]:
aa[]

0

In [69]:
max_epoch = 5

for i in range(max_epoch+1):
    model.fit(x_train, y_train,
            batch_size = 1000,
            epochs = 10,
            verbose = 1,
            validation_split=0.1)

    y_pred = model.predict(x_test)
    print(len(y_pred[(y_pred[:,0] > y_pred[:,1])]))
    print(len(y_pred[(y_pred[:,0] < y_pred[:,1])]))   # 연체자

    x_train = np.concatenate([x_train, x_overdue])
    y_train = np.concatenate([y_train, y_overdue])

Train on 72167 samples, validate on 8019 samples
Epoch 1/10
72167/72167 [==============================] - 1s - loss: 0.1764 - f1: 0.9575 - val_loss: 0.1871 - val_f1: 0.9540
Epoch 2/10
72167/72167 [==============================] - 1s - loss: 0.1763 - f1: 0.9574 - val_loss: 0.1871 - val_f1: 0.9540
Epoch 3/10
72167/72167 [==============================] - 1s - loss: 0.1749 - f1: 0.9575 - val_loss: 0.1857 - val_f1: 0.9540
Epoch 4/10
72167/72167 [==============================] - 1s - loss: 0.1740 - f1: 0.9575 - val_loss: 0.1853 - val_f1: 0.9540
Epoch 5/10
72167/72167 [==============================] - 1s - loss: 0.1725 - f1: 0.9575 - val_loss: 0.1830 - val_f1: 0.9540
Epoch 6/10
72167/72167 [==============================] - 1s - loss: 0.1706 - f1: 0.9575 - val_loss: 0.1840 - val_f1: 0.9540
Epoch 7/10
72167/72167 [==============================] - 1s - loss: 0.1687 - f1: 0.9575 - val_loss: 0.1779 - val_f1: 0.9540
Epoch 8/10
72167/72167 [==============================] - 1s - loss: 0.1649 

ValueError: all the input arrays must have same number of dimensions

In [ ]:
x_train.shape

In [ ]:
x_overdue.shape

In [ ]:
x_train = np.concatenate([x_train, x_overdue])
x_train.shape

In [ ]:
type(x_train)